During development of [NI Water](https://github.com/andrewbolster/bolster/pull/1009), the dataset changed underfoot and became disconnected from the [OpenDataNI dataset](https://admin.opendatani.gov.uk/dataset/ni-water-customer-tap-authorised-supply-point-results) that drove the mapping between Zones and Postcodes

Presumably, it's a load of postcodes (all BT34) that didn't participate in the 2023, but did in 2022, which doesn't make any sense either because the api call ("https://www.niwater.com/water-quality-lookup.ashx?z={zone_code}") is just returning blank. (Zone ZS0904)

Anyway, this is here just to remind me when this all goes to hell next time. 

In [2]:
from bolster.data_sources import ni_water

In [3]:
df = ni_water.get_water_quality()

In [4]:
df

,Water Supply Zone,Total Hardness (mg/l),Magnesium (mg/l),Potassium (mg/l),Calcium (mg/l),Total Hardness (mg CaCO3/l),Clark English Degrees,French Degrees,German Degrees,NI Hardness Classification,Dishwasher Setting
ZS1002,Carran Hill Camly,48.5,5.5,4.1,39.5,121.5,8.5,12.1,6.8,Slightly Hard,1 - 2
ZS0502,Poleglass Dunmurry,61.1,8.4,3.9,47.4,153.6,10.8,15.4,8.6,Moderately Hard,1 - 2
ZS0503,Forked Bridge Stoneyford,61.9,8.4,4.0,48.0,155.0,10.9,15.5,8.7,Moderately Hard,1 - 2
ZS0105,Belfast Breda South,52.3,7.2,3.2,40.2,130.6,9.1,13.1,7.3,Slightly Hard,1 - 2
ZN0202,Altnahinch Bushmills,38.5,4.5,0.7,31.0,96.3,6.7,9.6,5.4,Moderately Soft,0 - 1
ZS0814,Castor Bay Moira,62.1,8.4,3.9,48.3,155.8,10.9,15.6,8.7,Moderately Hard,1 - 2
ZS0405,Drumaroad Ards West,9.5,0.6,0.4,8.6,24.1,1.7,2.4,1.3,Soft,0
ZS0810,Castor Bay Tandragee,55.5,7.5,3.6,43.1,138.9,9.7,13.9,7.8,Slightly Hard,1 - 2
ZS0102,Dunore Ballygomartin South,60.1,8.5,3.8,46.3,151.2,10.6,15.1,8.5,Moderately Hard,1 - 2
ZN0607,Corrody Derry,37.8,3.3,1.3,32.5,95.0,6.7,9.5,5.3,Moderately Soft,0 - 1


In [5]:
maping = ni_water.get_postcode_to_water_supply_zone()

In [6]:
{ code for code, zone in maping.items() if zone == 'ZS0904'}

{'BT34 3EP',
 'BT34 1RG',
 'BT34 2PF',
 'BT34 3DD',
 'BT34 3NX',
 'BT34 1LJ',
 'BT34 3LW',
 'BT34 2PE',
 'BT34 2SU',
 'BT34 3LB',
 'BT34 5ER',
 'BT34 5BG',
 'BT34 3GL',
 'BT34 2RG',
 'BT34 4AT',
 'BT34 2HE',
 'BT34 5TW',
 'BT34 3LG',
 'BT34 5RN',
 'BT34 5AU',
 'BT34 4GQ',
 'BT34 4HB',
 'BT34 4AR',
 'BT34 4EJ',
 'BT34 4NR',
 'BT34 5DQ',
 'BT34 5YJ',
 'BT34 3BF',
 'BT34 1EP',
 'BT34 3HB',
 'BT34 4UE',
 'BT34 4RU',
 'BT34 5NB',
 'BT34 4AN',
 'BT34 3GZ',
 'BT34 5AB',
 'BT34 3AQ',
 'BT34 4ZG',
 'BT34 4FS',
 'BT34 1SG',
 'BT34 1DB',
 'BT34 5UE',
 'BT34 4TQ',
 'BT34 4SJ',
 'BT34 1JP',
 'BT34 2GX',
 'BT34 3NA',
 'BT34 1FE',
 'BT34 4BZ',
 'BT34 3PX',
 'BT34 3NY',
 'BT34 2LZ',
 'BT34 4GT',
 'BT34 3JZ',
 'BT34 1SR',
 'BT34 4UX',
 'BT34 2JT',
 'BT34 2SZ',
 'BT34 3XT',
 'BT34 2HT',
 'BT34 3EL',
 'BT34 1JH',
 'BT34 4EW',
 'BT34 2SP',
 'BT34 4UG',
 'BT34 3RR',
 'BT34 4SZ',
 'BT34 4EE',
 'BT34 1BY',
 'BT34 5UA',
 'BT34 5PZ',
 'BT34 1FH',
 'BT34 2FY',
 'BT34 2AT',
 'BT34 2JS',
 'BT34 5QW',
 'BT34 4NG',

In [7]:
data = ni_water.get_water_quality_by_zone('ZS0101')

In [9]:
data.index

Index(['Water Supply Zone', 'Total Hardness (mg/l)', 'Magnesium (mg/l)',
       'Potassium (mg/l)', 'Calcium (mg/l)', 'Total Hardness (mg CaCO3/l)',
       'Clark English Degrees', 'French Degrees', 'German Degrees',
       'NI Hardness Classification', 'Dishwasher Setting'],
      dtype='object', name=0)

In [11]:
df['NI Hardness Classification'].value_counts().keys()

CategoricalIndex(['Slightly Hard', 'Moderately Hard', 'Moderately Soft',
                  'Soft'],
                 categories=['Soft', 'Moderately Soft', 'Slightly Hard', 'Moderately Hard'], ordered=True, dtype='category', name='NI Hardness Classification')

In [14]:
from urllib.error import HTTPError

isinstance(HTTPError), BaseException)

TypeError: HTTPError.__init__() missing 3 required positional arguments: 'msg', 'hdrs', and 'fp'

In [17]:
_error_ = BaseException

try: 
    raise ValueError("Boo!")
except _error_ as e:
    print(e)

Boo!
